<a href="https://colab.research.google.com/github/visheshagrawal/Deep_Learning_Sequences/blob/master/bi_lstm_multi_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets,transforms
import torch.optim as optim
from torch.autograd import Variable
from torchsample.modules import ModuleTrainer

# tr=datasets.MNIST("",train=True,download=True,transform=transforms.Compose([transforms.ToTensor()]))
train = np.load('reduced_train.npz')
x_train = train['X_train']
# x_train = torch.from_numpy(x_train)
y_train = train['y_train']
# y_train = torch.from_numpy(y_train)
# mask_train = train['mask_train']
#print(x_train)


In [ ]:
print(y_train)

[6 6 6 ... 4 4 4]


In [ ]:
z_train = torch.utils.data.TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
print(z_train[0])

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor(6.))


In [ ]:
val = np.load('reduced_val.npz')
x_val = val['X_val']
#x_val = torch.from_numpy(x_val)
y_val = val['y_val']
#y_val = torch.from_numpy(y_val)
mask_val = val['mask_val']
print(x_val.shape)

(635, 400, 20)


In [ ]:
z_test = torch.utils.data.TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
print(z_test[0])

(tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.2400, 0.0000, 0.2700,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]), tensor(6.))


In [ ]:
train_loader= torch.utils.data.DataLoader(z_train,64,True)
#print(z_train)
#print(train_loader)
test_loader= torch.utils.data.DataLoader(z_test,64,False)

In [ ]:
batch_size = 64
seq_len = 400
n_feat = 20
n_hid = 30
n_class = 10
lr = 0.0025
n_filt = 10
drop_prob = 0.5

In [ ]:
for batch_idx,(data,target) in enumerate(train_loader):
  print(len(train_loader))
  data=data.view(400,-1,20)
  print(data.size())
  print(target.size())
  break


38
torch.Size([400, 64, 20])
torch.Size([64])


In [ ]:
for data,target in test_loader:
  print(data.view(64,-1).size())
  print(target.size())
  print(data.shape[0])
  print(len(test_loader))
  break

torch.Size([64, 8000])
torch.Size([64])
64
10


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim)

        # Readout layer
        self.fc = nn.Sequential(nn.Linear(12000,60),nn.ReLU(),nn.Linear(60,output_dim),nn.LogSoftmax(dim=1))

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(1), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(1), self.hidden_dim).requires_grad_()

        # 28 time steps8
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        #print(out.shape)
        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        #out = out.view(-1,12000)
        out=torch.reshape(out,(-1,12000))
        #print(out.shape)
        out = self.fc(out)
        # out.size() --> 100, 10
        return out

In [ ]:
class BLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(BLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.output_dim = output_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim,bidirectional=True)
        #self.fc = nn.Sequential(nn.Linear(hidden_dim*2,100),nn.ReLU(),nn.Linear(100,output_dim),nn.LogSoftmax(dim=1))
        #self.fc = nn.Linear(hidden_dim*2,output_dim)
        #self.fc = nn.Sequential(nn.Linear(hidden_dim*2,output_dim),nn.LogSoftmax(dim=1))
        self.fc = nn.Sequential(nn.Linear(48000,2000),nn.ReLU(),nn.Linear(2000,100),nn.ReLU(),nn.Linear(100,output_dim),nn.LogSoftmax(dim=1))

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim*2, x.size(1), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.layer_dim*2, x.size(1), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0, c0))
        #out = self.fc(out[-1,:,:])
        #out=torch.reshape(out,(-1,24000))
        out= self.fc(out.view(-1,48000))
        return out

In [ ]:
model=BLSTM(20,60,1,10)

criterion= nn.NLLLoss()

#criterion= nn.CrossEntropyLoss()

optimizer=optim.Adam(model.parameters(),lr=0.003)
#optimizer2=optim.Adam(fnnmodel.parameters(),lr=0.0025)

epochs=10

trainer = ModelTrainer(model)
  

In [ ]:
#model=nn.Sequential(nn.Linear(8000,n_hid),nn.ReLU(),nn.Linear(n_hid,n_class),nn.LogSoftmax(dim=1))

# lstmmodel=nn.LSTM(20,30,1)

# fnnmodel=nn.Sequential(nn.Linear(12000,30),nn.ReLU(),nn.Linear(30,n_class),nn.LogSoftmax(dim=1))

#model=LSTMModel(20,30,1,10)

model=BLSTM(20,60,1,10)

criterion= nn.NLLLoss()

#criterion= nn.CrossEntropyLoss()

optimizer=optim.Adam(model.parameters(),lr=0.003)
#optimizer2=optim.Adam(fnnmodel.parameters(),lr=0.0025)

epochs=10

for epoch in range(epochs):
  running_loss=0
  total_loss=0
  for batch_idx,(data,target) in enumerate(train_loader):
    target=target.type(torch.LongTensor)
    data, target = Variable(data), Variable(target)
    data=data.view(400,-1,20)
    sh=data.shape[1]
    #forward
    output = model(data)
    loss=criterion(output,target)
    
    #backward
    optimizer.zero_grad()
    loss.backward()
    
    #adam step
    optimizer.step()
    running_loss += loss.item()
    # if batch_idx % 10 == 0:
    # #if True:
    #   print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
    #   epoch, batch_idx * data.shape[1], len(train_loader.dataset),
    #   100. * batch_idx / len(train_loader), loss.item()))
    total_loss+=running_loss
    print(running_loss)
    print("Training epoch: "+str(epoch)+" loss for this batch is "+str(running_loss))
    running_loss=0


  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data=data.view(-1,8000)
    sh1=data.shape[0]
    target=target.type(torch.LongTensor)
    # h0 = torch.randn(1, sh1, 30)
    # c0 = torch.randn(1, sh1, 30)
    # hidden=(h0,c0)
    data=data.view(400,-1,20)
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    #output,hid=lstmmodel(data,hidden)
    #output=output.view(-1,12000)
    #fin_output=fnnmodel(output)
    #loss=criterion(fin_output,target)
    # sum up batch loss
    test_loss += criterion(output,target)
    # get the index of the max log-probability
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  #test_loss /= len(test_loader.dataset)
  test_loss /= batch_size
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))
  print(str(total_loss/(batch_idx+1)))
  print("Epoch finished")


2.2920620441436768
Training epoch: 0 loss for this batch is 2.2920620441436768
10.214472770690918
Training epoch: 0 loss for this batch is 10.214472770690918
10.706966400146484
Training epoch: 0 loss for this batch is 10.706966400146484
5.4987030029296875
Training epoch: 0 loss for this batch is 5.4987030029296875
3.5156853199005127
Training epoch: 0 loss for this batch is 3.5156853199005127
3.1979963779449463
Training epoch: 0 loss for this batch is 3.1979963779449463
2.4503705501556396
Training epoch: 0 loss for this batch is 2.4503705501556396
2.2560675144195557
Training epoch: 0 loss for this batch is 2.2560675144195557
2.206488609313965
Training epoch: 0 loss for this batch is 2.206488609313965
2.122464895248413
Training epoch: 0 loss for this batch is 2.122464895248413
2.2142844200134277
Training epoch: 0 loss for this batch is 2.2142844200134277
2.0483896732330322
Training epoch: 0 loss for this batch is 2.0483896732330322
2.073277235031128
Training epoch: 0 loss for this batch 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.3021, Accuracy: 187/635 (29%)

2.648063440071909
Epoch finished
1.933566927909851
Training epoch: 1 loss for this batch is 1.933566927909851
2.0640337467193604
Training epoch: 1 loss for this batch is 2.0640337467193604
1.9009298086166382
Training epoch: 1 loss for this batch is 1.9009298086166382
1.8503166437149048
Training epoch: 1 loss for this batch is 1.8503166437149048
1.9655139446258545
Training epoch: 1 loss for this batch is 1.9655139446258545
1.915433406829834
Training epoch: 1 loss for this batch is 1.915433406829834
1.8886629343032837
Training epoch: 1 loss for this batch is 1.8886629343032837
1.757684588432312
Training epoch: 1 loss for this batch is 1.757684588432312
1.941969633102417
Training epoch: 1 loss for this batch is 1.941969633102417
1.7436027526855469
Training epoch: 1 loss for this batch is 1.7436027526855469
1.9631675481796265
Training epoch: 1 loss for this batch is 1.9631675481796265
1.7343627214431763
Training epoch: 1 loss for th

In [ ]:
def check_accuracy(loader, model):
    # if loader.dataset.train:
    #     print("Checking accuracy on training data")
    # else:
    #     print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.squeeze(1)
            y = y

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy  \
              {float(num_correct)/float(num_samples)*100:.2f}"
        )
    model.train()


In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Got 2360 / 2423 with accuracy                97.40
Got 453 / 635 with accuracy                71.34
